In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=a42791ddaafed85424c43e2fa79f39ac2cbcc7f29cda575ee3ca71f015d474db
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Initialize Spark Session
spark = SparkSession.builder.appName("HotelBookingClassification").getOrCreate()

# Load the dataset
data = spark.read.csv("/content/merged_file.csv", header=True, inferSchema=True)

# Index categorical features (String to Numeric) with handleInvalid='skip'
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid='skip').fit(data)
            for column in ['deposit_type', 'country', 'customer_type', 'hotel', 'meal', 'distribution_channel', 'reserved_room_type', 'assigned_room_type']]  # Include all string columns here

# Apply StringIndexer to convert categorical variables
for indexer in indexers:
    data = indexer.transform(data)

# Selecting the top features
selected_features = [
    "hotel_index",  # Use indexed columns
    "lead_time",
    "stays_in_weekend_nights",
    "stays_in_week_nights",
    "adults",
    "children",
    "babies",
    "meal_index",  # Use indexed columns
    "country_index",  # Use indexed columns
    "distribution_channel_index",  # Use indexed columns
    "is_repeated_guest",
    "previous_cancellations",
    "previous_bookings_not_canceled",
    "reserved_room_type_index",  # Use indexed columns
    "assigned_room_type_index",  # Use indexed columns
    "booking_changes",
    "deposit_type_index",  # Use indexed columns
    "days_in_waiting_list",
    "customer_type_index",  # Use indexed columns
    "adr",
    "required_car_parking_spaces",
    "total_of_special_requests",
    "Distribution_Id"
]

# Assemble features into a single vector
assembler = VectorAssembler(inputCols=selected_features, outputCol="features")
data = assembler.transform(data)

# Standardizing the features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(data)
data = scaler_model.transform(data)

# Select only necessary columns
data = data.select("scaledFeatures", col("is_canceled").alias("label"))

# Train-test split
train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)

from pyspark.ml.classification import LinearSVC

# Train SVM model
svm = LinearSVC(featuresCol='scaledFeatures', labelCol='label', maxIter=100)
svm_model = svm.fit(train_data)

# Make predictions
predictions = svm_model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
svm_accuracy = evaluator.evaluate(predictions)
print(f"Support Vector Machine Accuracy: {svm_accuracy}")


Support Vector Machine Accuracy: 0.7646084760033679
